# DAWN Winter School 2022 
### - EAZY Tutorial

This tutorial will show and provide the basics of fitting photometric data with the SED-fitting code EAZY (Brammer et al. 2008). In this example, we will be fitting the Hubble Deep Field North (HDFN) catalog provided with the original EAZY code.

This tutorial will mostly rely on eazy-py (https://github.com/gbrammer/eazy-py).

This tutorial will focus on fitting photometry to derive photometric redshifts and physical quantities, which we will explore to understand the targets in our catalog. We will use different selection criteria, look for interesting objects in the catalog, amongst other things.

### Demo fitting the HDFN catalog provided with the original EAZY code

In [1]:
%matplotlib inline

In [2]:
# environment installation for hosted notebooks
# (e.g., mybinder, GoogleCollab)
import os
orig_wd = os.getcwd()

try:
    import eazy
    HAS_EAZY = True
except:
    HAS_EAZY = False

# mybinder
if ('jovyan' in orig_wd):
    if not HAS_EAZY:
        print('Install on /home/jovyan')

        os.chdir('/home/jovyan')
        !pip install . -r requirements.txt

        os.chdir(orig_wd)
    
    try:
        import grizli
    except:
        !pip install cython
        !pip install git+https://github.com/gbrammer/grizli
            
    try:
        print('EAZYCODE = '+os.getenv('EAZYCODE'))
    except:
        %env EAZYCODE=/home/jovyan/eazy-photoz/

# Google collab
if ('/content' in orig_wd):
    if not HAS_EAZY:
        print('Install on /content (Google Collab')

        os.chdir('/content')
        !git clone https://github.com/gbrammer/eazy-py.git --recurse-submodule
        os.chdir('/content/eazy-py')
        !pip install . -r requirements.txt

        os.chdir(orig_wd)
    
    try:
        import grizli
    except:
        !pip install cython
        !pip install git+https://github.com/gbrammer/grizli
            
    try:
        print('EAZYCODE = '+os.getenv('EAZYCODE'))
    except:
        %env EAZYCODE=/content/eazy-py/eazy-photoz/


In [3]:
# Module versions
import importlib
import sys
import time
print(time.ctime() + '\n')

print(sys.version + '\n')

for module in ['numpy', 'scipy', 'matplotlib','astropy','eazy', 'prospect']:
    #print(module)
    mod = importlib.import_module(module)
    print('{0:>20} : {1}'.format(module, mod.__version__))

Fri Feb  4 12:58:24 2022

3.6.10 |Anaconda, Inc.| (default, May  7 2020, 23:06:31) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]

               numpy : 1.19.2
               scipy : 1.5.2
          matplotlib : 3.3.1
             astropy : 4.0.1.post1
                eazy : 0.4-3-gd86432e
            prospect : 1.0.0


In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt

import eazy

# Symlink templates & filters from the eazy-code repository
try:
    print('EAZYCODE = '+os.getenv('EAZYCODE'))
except:
    pass

if not os.path.exists('templates'):
    eazy.symlink_eazy_inputs() 

EAZYCODE = /Users/arteaga/eazy-py/eazy-photoz/


In [7]:
# quiet numpy/astropy warnings
import warnings
from astropy.utils.exceptions import AstropyWarning

np.seterr(all='ignore')
warnings.simplefilter('ignore', category=AstropyWarning)

### Fit Input Parameters

Default parameters are stored in the file `eazy/data/zphot.param.default` in the repository.

The parameters are necessary to fit the catalog, here one specifies the catalog file, the templates to be used in the fitting, the redshift grid, etc. The user input is transparent and easy to implement. The parameters that differ from defaults can be provided in a dictionary as shown below.

In [8]:
params = {}

# Catalog file
params['CATALOG_FILE'] = os.path.join(os.getenv('EAZYCODE'), 'inputs/hdfn_fs99_eazy.cat')

params['MAIN_OUTPUT_FILE'] = 'hdfn.eazypy'

# Galactic extinction
params['MW_EBV'] = 0.0103
params['CAT_HAS_EXTCORR'] = True

# Redshift grid (minimum, maximum and grid step)
params['Z_STEP'] = 0.01
params['Z_MIN'] = 0.01
params['Z_MAX'] = 6.

# Prior files
params['PRIOR_ABZP'] = 25 
params['PRIOR_FILTER'] = 28 # K
params['PRIOR_FILE'] = 'templates/prior_K_TAO.dat'

# Template set to fit photometry
params['TEMPLATES_FILE'] = 'templates/fsps_full/tweak_fsps_QSF_12_v3.param'

# Fix spectroscopic redshift
params['FIX_ZSPEC'] = False

# IGM parameters
params['IGM_SCALE_TAU'] = 1.0

In [9]:
# The translate file will read in the filter bandpasses from the catalog
translate_file = os.path.join(os.getenv('EAZYCODE'), 'inputs/zphot.translate')

# Here we read in the catalog, templates and filters
self = eazy.photoz.PhotoZ(param_file=None, translate_file=translate_file, zeropoint_file=None, 
                          params=params, load_prior=True, load_products=False)

Read default param file: /Users/arteaga/miniconda3/envs/grizli-dev/lib/python3.6/site-packages/eazy/data/zphot.param.default
Read CATALOG_FILE: /Users/arteaga/eazy-py/eazy-photoz/inputs/hdfn_fs99_eazy.cat
   >>> NOBJ = 1067
f_f300w e_f300w ( 10): hst/wfpc2_f300w.dat
f_f450w e_f450w ( 12): hst/wfpc2_f450w.dat
f_f606w e_f606w ( 14): hst/wfpc2_f606w.dat
f_f814w e_f814w ( 16): hst/wfpc2_f814w.dat
f_irimj e_irimj ( 26): KPNO/IRIMJ
f_irimh e_irimh ( 27): KPNO/IRIMH
f_irimk e_irimk ( 28): KPNO/IRIMK
Read PRIOR_FILE:  templates/prior_K_TAO.dat
Template grid: templates/fsps_full/tweak_fsps_QSF_12_v3.param (this may take some time)
Process template tweak_fsps_QSF_12_v3_001.dat (NZ=1).
Process template tweak_fsps_QSF_12_v3_002.dat (NZ=1).
Process template tweak_fsps_QSF_12_v3_003.dat (NZ=1).
Process template tweak_fsps_QSF_12_v3_004.dat (NZ=1).
Process template tweak_fsps_QSF_12_v3_005.dat (NZ=1).
Process template tweak_fsps_QSF_12_v3_006.dat (NZ=1).
Process template tweak_fsps_QSF_12_v3_007.dat 

### Now we fit the whole catalog

With the command `self.fit_catalog`, we fit the input catalog.

In [12]:
# Turn off error corrections derived above
self.set_sys_err(positive=True)

# Full catalog
sample = np.isfinite(self.cat['z_spec'])

# fit_parallel renamed to fit_catalog 14 May 2021
self.fit_catalog(self.idx[sample], n_proc=8)

AttributeError: 'PhotoZ' object has no attribute 'fit_catalog'

In [ ]:
# Show zspec-zphot comparison
fig = self.zphot_zspec()

### Derived parameters (z params, RF colors, masses, SFR, etc.)

With `self.standard_output` we can obtain the derived parameters from the fitting, this includes rest-frame colours, star formation rates, photometric redshifts, stellar masses, etc.

In [ ]:
warnings.simplefilter('ignore', category=RuntimeWarning)
zout, hdu = self.standard_output(rf_pad_width=0.5, rf_max_err=2, 
                                 prior=True, beta_prior=True)

# 'zout' also saved to [MAIN_OUTPUT_FILE].zout.fits

In [ ]:
# the 'zout' object has all the derived physical properties, which can be seen by using the command below:
zout.colnames

In [ ]:
# it also contains relevant information about the fit performed, templates used, etc. 
# this information can be accessed
zout.meta

## Exploring the output

Once we have fitted our catalog with EAZY and derived multiple physical parameters, we can explore the properties of our sample, as well as individual objects. Here we show some examples of different analyses that one can perform with the EAZY output.

### UVJ diagram

Part of what makes understanding galaxy evolution so challenging is the wealth of galaxy properties. Explaining the evolution of all these properties and the correlation between them is a daunting task. Two-colour diagrams have been shown to be a useful and simple mechanism for comprehending astronomical objects, in particular the $UVJ$ diagram, which is obtained by plotting the rest-frame $U − V$ versus $V − J$ colours.

The $UVJ$ diagram separates quiescent from star forming galaxies (SFGs). It is particularly useful due to its ability to break the degeneracy between age and reddening for red galaxies: in most cases, galaxies with blue UV colours display rather dust-free star formation activity, whereas galaxies can appear red due to dust obscuration or being quiescent, since evolved older stellar populations will dominate. On the other hand, quiescent galaxies unobscured by dust are blue in $VJ$, therefore they will be in a different position than the SFGs in the $UVJ$ plane, leaving the two types of sources empirically set apart. Williams et al., 2009 (https://arxiv.org/abs/0806.0625), determined a border between the two regions that is still relevant and is being used in the present-day.

In [ ]:
# Show UVJ diagram
uv = -2.5*np.log10(zout['restU']/zout['restV'])
vj = -2.5*np.log10(zout['restV']/zout['restJ'])
ssfr = zout['sfr']/zout['mass']

sel = (zout['z_phot'] > 0.2) & (zout['z_phot'] < 1)
plt.scatter(vj[sel], uv[sel], c=np.log10(ssfr)[sel], 
            vmin=-13, vmax=-8, alpha=0.5)

plt.xlim(-0.2, 2.3); plt.ylim(0, 2.4); plt.grid()
plt.xlabel(r'$(V-J)_0$'); plt.ylabel(r'$(U-V)_0$')    

As can be seen below, the $UVJ$ diagram can separate quiescent galaxies from star-forming galaxies, and moreover, break the degeneracy with dusty-star forming galaxies (which lay on the upper right end of the diagram).

**[we are missing a plot here]**
![UVJ]()



### Selecting bright objects

Another exciting avenue is to select interesting objects, such as the brightest objects with z_spec > 1. The command `self.show_fit` allows us to plot the resulting SED-fit, as well as the redshift probability density function, where we can see the spectroscopic redshift and the photo-z PDF.

Besides this, the `self.cat` contains the whole catalog, and can be used to perform different selections based on various criteria.

In [ ]:
# Show brightest objects with z_spec > 1
imag = params['PRIOR_ABZP'] - 2.5*np.log10(self.cat['f_f814w'])
sel = (self.cat['z_spec'] > 1.1)

so = np.argsort(imag[sel])
ids = self.cat['id'][sel][so]

for i in range(4):
    fig, data = self.show_fit(ids[i], xlim=[0.2, 3], show_components=True,
                              logpz=True, zr=[0,4])

## Concluding Remarks

This turorial has hopefully taught you to extract and derive basic galaxy properties from photometric data, including fitting photometric redshifts, exploring the UVJ diagram and its implications, analysing galaxy spectral energy distributions and selecting interesting objects from a large catalog.


# Proposed Exercises

1. Find the most massive galaxy in the sample and the highest redshift candidate.

---

2. Explore the UVJ diagram, and make a sub-sample of quiescent galaxies according to the Williams+09 (https://arxiv.org/pdf/0806.0625.pdf) selection criteria.

(U −V) > 0.88×(V −J)+0.69

(U −V) > 0.88×(V −J)+0.59

(U −V) > 0.88×(V −J)+0.49

Additional criteria of U − V > 1.3 and V − J < 1.6 are applied to the quiescent galaxies at all redshifts to prevent contamination from unobscured and dusty star–forming galaxies, respectively.

---

3. Select the object with ID=**XXX [@Kate, find a dusty star forming galaxy with high Av that falls in the quiescent box but is actually star forming]**, and compare its spectral energy distribution (SED) with some SEDs from your previous quiescent galaxy population selection. Do they look the same? If not, why? You can also check its physical properties such as SFR and mass. Do you think the object with ID=**XXX** is a quiescent galaxy? If not, what kind of galaxy do you think it is? 

---

4. Plot the PDF(z) of a quiescent galaxy, a blue star-forming galaxy and a red star-forming galaxy (you can use `self.show_fit` with `logpz=True`). Are there differences in the uncertainty on the photo-z for the different populations?

---

5. Try to find the most interesting object by your own liking/criteria (think about what most interesting could involve - brightest? highest mass? highest redshift? most quiescent?)

---

## Extra exercise

6. Explore the redshift vs K-band relation and the redshift vs Mass for the whole catalog. What can you tell about completeness of the sample? **[Maybe cite some references here so they know what to look for?]**